In [1]:
import sys
sys.path.append('..')

In [2]:
import tagnews

In [3]:
df = tagnews.load_data()

..\tagnews\utils\load_data.py:194: RuntimeWarning: 1 location strings were not found in the bodytext.
  RuntimeWarning)


In [4]:
import pandas as pd

In [5]:
time = pd.to_datetime(df['created'])
df = df.loc[('2015' < time) & (time < '2016')]

In [6]:
crimetags = tagnews.CrimeTags()
geoextractor = tagnews.GeoCoder()

In [7]:
preds = df['bodytext'].apply(crimetags.tagtext_proba)

In [8]:
preds.shape

(34531, 38)

In [9]:
preds.columns

Index(['ARSN', 'BEAT', 'BURG', 'CCCC', 'CCJ', 'CCSP', 'CPBD', 'CPD', 'CPLY',
       'CPS', 'CPUB', 'DOMV', 'DRUG', 'DUI', 'ENVI', 'FRUD', 'GANG', 'GLBTQ',
       'GUNV', 'HOMI', 'IDOC', 'ILSC', 'ILSP', 'IMMG', 'IPRA', 'JUVE', 'OEMC',
       'PARL', 'POLB', 'POLM', 'PROB', 'REEN', 'ROBB', 'SAO', 'SEXA', 'TASR',
       'UNSPC', 'VIOL'],
      dtype='object')

In [10]:
(preds['SEXA'] > 0.7).sum()

313

In [12]:
geostrings = df.loc[preds['SEXA'] > 0.7, 'bodytext'].apply(geoextractor.extract_geostrings)

In [13]:
sum(geostrings.apply(len))

217

In [24]:
lat_longs = []
for i, gs in enumerate(geostrings):
    lat_longs.append(geoextractor.lat_longs_from_geostring_lists(gs, sleep_secs=5))
    if not (i % 30):
        print(i / geostrings.size)

0.0
0.09584664536741214
0.19169329073482427
0.28753993610223644
0.38338658146964855
0.4792332268370607
0.5750798722044729
0.670926517571885


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=Larkin+Chicago+Illinois&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=south+Chicago+Illinois&limitnbresult=1&format=json


0.7667731629392971


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=700+block+of+Patton+Circle&limitnbresult=1&format=json
Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=block+Chicago+Illinois&limitnbresult=1&format=json


0.8626198083067093


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=3500+block+of+West&limitnbresult=1&format=json


0.9584664536741214


Status code 401 from https://services.gisgraphy.com/geocoding/: ERROR - 401 Client Error: Unauthorized for url: https://services.gisgraphy.com/geocoding/?address=Kingdom&limitnbresult=1&format=json


In [16]:
import numpy as np

In [25]:
import folium

from folium.plugins import HeatMap

In [39]:
coords = [x for ll in lat_longs for x in ll[0] if x is not None]

In [44]:
pd.DataFrame(coords, columns=['lat', 'long']).to_csv('sexual-assault-2015-lat-longs.csv')

In [43]:
m = folium.Map(location=[41.87871, -87.6298])

for coord in coords:
    folium.Marker(coord).add_to(m)

# HeatMap(np.unique(coords, axis=0).tolist()).add_to(m)

m